In [1]:
import pyomo.environ as pe
import pandas as pd
import numpy as np

## Data Preparation

In [2]:
D = pd.read_csv("Data\d_matrix.csv", index_col = 0)
# g: distance with SFA
adist = D.iloc[-2]

# convert g to g'
g = adist.apply(lambda x: max(5, x))

In [3]:
D2 = pd.DataFrame()

In [4]:
for i in D.columns:
    D2[i] = D[i].apply(lambda x: max(x, 5) if x>0 else (-70/4.5 + 5))

In [9]:
D3 = pd.DataFrame()
for i in D.columns:
    D3[i] = D[i].apply(lambda x: max(x, 5) if x>0 else (-30/4 + 5))

In [5]:
demand = pd.read_excel("Data\demand.xlsx", index_col = 0)

d = demand.demand

In [10]:
a = pd.DataFrame(columns = d.index, index = d.index)

for n in range(1, 21):
    cname = "cluster" + str(n)
    path = "20abc\\" + cname + "result20bc_node.csv"
    result = pd.read_csv(path, index_col = 0)
    nodes = result.index
    for i in nodes:
        for j in nodes:
            a.loc[i][j] = result.loc[i][j]  

In [11]:
demand_matrix = a.apply(lambda x: x*d)

hub_d = demand_matrix.sum()
hub_d = hub_d[hub_d > 0]

In [12]:
hubs = list(hub_d.index)

In [13]:
len(hubs)

151

### Model

In [19]:
def gsolver(node, d):
    model1 = pe.ConcreteModel()
    model1.x = pe.Var(node, node, domain = pe.Binary)
    model1.y = pe.Var(node, domain = pe.Binary)
    model1.a1 = pe.Var(node, domain=pe.NonNegativeIntegers)
    model1.a2 = pe.Var(node, domain=pe.NonNegativeIntegers)
    model1.b1 = pe.Var(node, domain=pe.NonNegativeIntegers)
    model1.b2 = pe.Var(node, domain=pe.NonNegativeIntegers)

    model1.costs = pe.Objective(expr = 
        sum(model1.x[i,j] * (70+ 4.5* (D2.loc[i,j]-5)) * model1.a2[i] for i in node for j in node) + 
        sum(model1.y[j] * (70+4.5*(g[j]-5)) * model1.a1[j] for j in node) +
        sum(model1.x[i,j] * (30+ 4* (D3.loc[i,j]-5)) * model1.b2[i] for i in node for j in node) + 
        sum(model1.y[j] * (30+4*(g[j]-5)) * model1.b1[j] for j in node),
        sense = pe.minimize)

    # A node can only be assigned to a hub node
        
    def rule_1(mod, i, j):
        return mod.x[i,j] <= mod.y[j]
    model1.const1 = pe.Constraint(node, node, rule = rule_1)

    # Each node is assigned to 1 and only 1 hub
    def rule_2(mod, i):
        return sum(mod.x[i, j] for j in node) == 1
    model1.const2 = pe.Constraint(node, rule = rule_2)

    # Number of Type A Vans -2
    def rule_3(mod, i):
        return 800 * mod.a2[i] + 200 * mod.b2[i]>= d[i]
    model1.const3 = pe.Constraint(node, rule = rule_3)

    # Number of Type A Vans
    def rule_4(mod, j):
        return 800 * mod.a1[j] + 200 * mod.b1[j]>= sum(mod.x[i,j] * d[i] for i in node)
    model1.const4 = pe.Constraint(node, rule = rule_4)

    time_limit = 10*60
    solver = pe.SolverFactory('gurobi')
    solver.options['TimeLimit'] = time_limit
    result = solver.solve(model1, tee = True)
    
    return model1

In [15]:
def result(node,model1):
    c = model1.costs()
    c = round(c,2)
    
    hub = []
    avan = []
    for j in node:
        if round(model1.y[j]()) == 1:
            hub.append(j)
            avan.append(model1.a1[j]())
    hubs = pd.DataFrame({"1st_Level":hub, "TypeA":avan})
    
    df = pd.DataFrame(columns = node, index = node)
    for i in node:
        for j in node:
            df.loc[i][j] = round(model1.x[i,j]())
    
    hubs.to_csv("first_level_ab.csv", index = 0)
    df.to_csv("second_level_ab.csv")
    
    return c, hubs, df

In [20]:
%%time
model1 = gsolver(hubs, hub_d)

Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmp68w369vn.pyomo.lp
Reading time = 0.24 seconds
x23557: 23255 rows, 23557 columns, 91809 nonzeros
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 23255 rows, 23557 columns and 91809 nonzeros
Model fingerprint: 0x54b6c296
Model has 45602 quadratic objective terms
Variable types: 1 continuous, 23556 integer (22952 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e+01, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective 1.699291e+14
Presolve removed 1 rows and 1 columns
Presolve time: 0.3

 10363  4836 9945.86350  208  153 26238.1210 9833.97100  62.5%  33.7  422s
 10535  4957 10164.8965  219  153 26238.1210 9833.97100  62.5%  33.2  425s
 10887  5188 10149.6685  243  152 26238.1210 9833.97100  62.5%  32.2  431s
 11225  5428 10221.6910  268  152 26238.1210 9833.97100  62.5%  31.2  437s
 11415  5555 10120.5715  282  152 26238.1210 9833.97100  62.5%  30.8  441s
 11809  5827 10124.8150  311  152 26238.1210 9833.97100  62.5%  29.8  448s
 12008  5932 10023.9855  326  153 26238.1210 9833.97100  62.5%  29.3  452s
 12186  6092 10047.4980  339  153 26238.1210 9833.97100  62.5%  28.9  457s
 12413  6231 10209.0820  356  151 26238.1210 9833.97100  62.5%  28.4  461s
 12641  6395 10099.2435  372  152 26238.1210 9833.97100  62.5%  27.9  465s
 12897  6555 10123.7595  391  152 26238.1210 9833.97100  62.5%  27.4  470s
 13145  6746 10282.9945  409  150 26238.1210 9833.97100  62.5%  26.9  476s
 13419  6946 10269.9355  429  150 26238.1210 9833.97100  62.5%  26.4  482s
 13710  7150 10214.9340  

In [18]:
c, high, df = result(hubs, model1)
print(c)

26266.59
